In [ ]:
%pip install datasets tokenizers
# !huggingface-cli login

In [ ]:
from datasets import load_dataset
load_dataset("balochiml/balochi-language-data", data_dir="data", cache_dir="../data")

# Generate the processed data without English characters

In [ ]:
#download dataset
!sh download.sh

In [ ]:
import os


def get_txt_file_paths(directory):
    txt_file_paths = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".txt"):
                file_path = os.path.join(root, file)
                txt_file_paths.append(file_path)
    return txt_file_paths


# Replace "directory_path" with the actual path of the directory you want to search
directory_path = "../data/raw_text"
txt_paths = get_txt_file_paths(directory_path)

len(txt_paths)

In [ ]:
import re


def clean_text(file_path):
    # Open the file and read it into memory
    with open(file_path, "r", encoding="utf-8") as file:
        text = file.read()

    # Remove English-language characters and numbers
    text = re.sub(r"[a-zA-Z0-9]", "", text)

    # Remove any excess whitespace
    text = re.sub(r"[^\S\n]+", " ", text)

    return text

In [ ]:
for path in txt_paths:
    cleaned_text = clean_text(path)

    # write the cleaned text to a new file with an incremented filename
    # write the files all into the '../data/processed_text' directory
    with open(
        f'../data/processed_text/{path.split("/")[-1]}', "w", encoding="utf-8"
    ) as file:
        file.write(cleaned_text)

# Training a Tokenizer using 🤗 Tokenizers

In [ ]:
from tokenizers import Tokenizer
from tokenizers.models import BPE

tokenizer = Tokenizer(BPE(unk_token="[UNK]"))

In [ ]:
from tokenizers.pre_tokenizers import Whitespace

tokenizer.pre_tokenizer = Whitespace()

In [ ]:
from tokenizers.trainers import BpeTrainer

trainer = BpeTrainer(
    min_frequency=2,
    vocab_size=30000,
    special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"],
    show_progress=True,
)

In [ ]:
# get a list of all the txt files in
# '/Users/strickvl/balochi/balochi-tokenizer/data/processed_text'

processed_files = get_txt_file_paths("../data/processed_text")
assert len(processed_files) == len(txt_paths)
len(processed_files)

In [ ]:
tokenizer.train(processed_files, trainer)

In [ ]:
tokenizer.model

In [ ]:
tokenizer.get_vocab_size()

In [ ]:
# tokenizer.get_vocab()

In [ ]:
tokenizer.save("../models/30k-balochi-tokenizer.json")

In [ ]:
tokenizer = Tokenizer.from_file("../models/30k-balochi-tokenizer.json")

In [ ]:
sample_text = "      آیک  جناورے اَت۔  لھتے گشیت آ سکیں کارزوالے ات کہ اگاں آزاتی دیگ بہ بیت، بازارءَ، لوگے ءَ، جاگاہ یے  ءَ،دپتر ء ُ کارگس یے  ءَ یا ھر ھما جاگاہ ءَ کہ شُت کنت مزنیں کارزوالی کنت۔گوں ھر کس ءَ جنگ ء ُ مڑ بیت۔گدء ُ پچاں  چنڈ چنڈ ء ُ راڑ راڑ کنت،کاگد ء ُ وانگیاں وارت ء ُ آدراہ کنت۔ورگی چیزاں اگاں وارت نکنت آھاں گٹ پاچیت ھراب کنت۔ایندگہ جناور چہ بندات ء َ ایشی ءِ کازوالیاں چہ وتا دیر دارگ ءِ کوشست کن اَنت۔ چیا کہ آ بازیں دگہ ھرابی ء ُ کارزوالی ھم کنت،پمیشکا کسانیں جناور  بالی مُرگ،کوہ پاچن،آسک ء ُ ایندگہ کسان کسانیں جناورچر آئی ءِ کارزوالیانی سوب ءَ آئی ءَ چہ سک باز شزار اَنت ۔".replace(
    "\xa0", ""
)
sample_sentence = sample_text.split("۔")[2]
sample_sentence

In [ ]:
tokenizer.encode(sample_sentence).tokens

# Training a custom tokenizer using Spacy and FastAI

In [ ]:
from fastai.text.all import *
files = get_text_files("../data/processed_text")

In [ ]:
len(files)

In [ ]:
txt = files[0].open().read(); txt[:75]

In [ ]:
spacy = WordTokenizer()
toks = first(spacy([txt]))
print(coll_repr(toks, 30))

In [ ]:
tkn = Tokenizer(spacy)
print(coll_repr(tkn(txt), 31))

In [ ]:
txts = L(o.open().read() for o in files)

In [ ]:
def subword(size: int):
    sp = SubwordTokenizer(vocab_sz=size)
    sp.setup(txts)
    return " ".join(first(sp([txt]))[:40])


In [ ]:
subword(1000)

In [ ]:
subword(275)

In [ ]:
toks200 = txts[:200].map(tkn)
toks200[0]

In [ ]:
num = Numericalize()
num.setup(toks200)
coll_repr(num.vocab,50)

In [ ]:
nums = num(toks)[:20]; nums

In [ ]:
' '.join(num.vocab[o] for o in nums)